In [1]:
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import AveragePooling2D,Flatten,Dropout,Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

C:\Users\mugesh\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\mugesh\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\mugesh\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\mugesh\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [3]:
ROOT_DIR = 'dataset/'

In [4]:
os.listdir(ROOT_DIR)

['without_mask', 'with_mask']

### *Preprocessing the Data*

In [5]:
image_data = []
label = []
for categories in os.listdir(ROOT_DIR):
    path = os.path.join(ROOT_DIR, categories)
    for file_name in os.listdir(path):
        file_name = os.path.join(path,file_name)
        image = plt.imread(file_name)# Reading the image file
        #if image is not None: # checking the image has data or not  
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        image = cv2.resize(image,(224,224)) # resize the image
        image = preprocess_input(image) # preprocessing the image so that model can take data as input
        # Storing the processed data into the list
        image_data.append(image)
        label.append(categories)

In [6]:
encoder = LabelEncoder()
label = encoder.fit_transform(label) # encoding the string into integers 

In [7]:
label = to_categorical(label) # one-hot encoding 

In [8]:
# converting the list of data into array 
image_data = np.array(image_data, dtype="float32")

### *Data is prepared building the model architecture*

In [9]:
mobilenet = MobileNetV2(weights="imagenet", include_top=False,input_shape=(224, 224, 3))
# loop over all layers in the mobilenet and freeze them so they will *not* be updated during the first training process
for layer in mobilenet.layers:
    layer.trainable = False 
# Adding the final layers
op_layer = mobilenet.output
avg_pool = AveragePooling2D(pool_size=(7, 7))(op_layer)
flatten = Flatten(name="flatten")(avg_pool)
h_layer1 = Dense(128, activation="relu")(flatten)
drop_layer = Dropout(0.5)(h_layer1)
out = Dense(2, activation="softmax")(drop_layer)
# merging the inputs and outputs as a single model
model = Model(inputs=mobilenet.input, outputs=out)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
______________________________________________________________________________________________

###### Compiling the model

In [10]:
adam = Adam(lr=1e-4, decay=1e-4/20)
model.compile(loss="binary_crossentropy", optimizer=adam, metrics=["accuracy"])

### *Spliting the data*

In [11]:
(trainX, testX, trainY, testY) = train_test_split(image_data, label, test_size=0.20, stratify=label, random_state=42)

### *Augmentation*

In [12]:
augment = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

### *Training the Model*

In [13]:
model.fit(
    augment.flow(trainX, trainY, batch_size=32),
    steps_per_epoch=len(trainX) // 32,
    validation_data=(testX, testY),
    validation_steps=len(testX) // 32,
    epochs=20)

Train for 95 steps, validate on 767 samples
Epoch 1/20
95/95 [==============================] - 65s 681ms/step - loss: 0.5222 - accuracy: 0.7278 - val_loss: 0.4265 - val_accuracy: 0.7541
Epoch 2/20
95/95 [==============================] - 58s 615ms/step - loss: 0.2938 - accuracy: 0.8823 - val_loss: 0.3194 - val_accuracy: 0.8505
Epoch 3/20
95/95 [==============================] - 47s 497ms/step - loss: 0.2410 - accuracy: 0.9080 - val_loss: 0.3280 - val_accuracy: 0.8424
Epoch 4/20
95/95 [==============================] - 46s 486ms/step - loss: 0.2159 - accuracy: 0.9192 - val_loss: 0.2869 - val_accuracy: 0.8696
Epoch 5/20
95/95 [==============================] - 48s 507ms/step - loss: 0.1888 - accuracy: 0.9314 - val_loss: 0.2151 - val_accuracy: 0.9090
Epoch 6/20
95/95 [==============================] - 49s 520ms/step - loss: 0.1771 - accuracy: 0.9351 - val_loss: 0.2539 - val_accuracy: 0.8967
Epoch 7/20
95/95 [==============================] - 46s 485ms/step - loss: 0.1457 - accuracy: 0.94

In [14]:
model.save('masknet.h5')